## Zhengxu Wang 
zhengxu@bu.edu  
cs505 hw2
## Task1
Getting fishing and football data from tweet.

In [1]:
import tweepy
import csv

client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGl1hAEAAAAAvHDZpENUinmoaM3e1EdQTyxEbtI%3D2SjBFtQBWxin1ZHoIXazuE22fNEvzdO3D3aTnzhfLS4pr1HWI3')
query = 'fishing lang:en -has:mentions -has:links -is:retweet'
tweets = list(tweepy.Paginator(client.search_recent_tweets, query=query, tweet_fields='created_at', max_results=100).flatten(limit=10000))

driveFolderDirectory = './'
savedFileName = 'tweetsFishing.csv'
pathToSave = driveFolderDirectory + savedFileName

with open(pathToSave, 'w', newline='') as csvfile:
  fieldnames = ['idx','tweetId', 'tweetText']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()
  for i,tweet in enumerate(tweets):
    writer.writerow({
        'idx': i,
        'tweetId': tweet.id,
        'tweetText': tweet.data['text']
        })

In [2]:
query = 'football lang:en -has:mentions -has:links -is:retweet'
tweets = list(tweepy.Paginator(client.search_recent_tweets, query=query, tweet_fields='created_at', max_results=100).flatten(limit=10000))


driveFolderDirectory = './'
savedFileName = 'tweetsFootball.csv'
pathToSave = driveFolderDirectory + savedFileName

with open(pathToSave, 'w', newline='') as csvfile:
  fieldnames = ['idx','tweetId', 'tweetText']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()
  for i,tweet in enumerate(tweets):
    writer.writerow({
        'idx': i,
        'tweetId': tweet.id,
        'tweetText': tweet.data['text']
        })

Loading data

In [3]:
import csv

def loadTextFromCSV(csvPath):
  tweetDict = {}
  with open(csvPath, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      tweetDict[int(row['idx'])] = row['tweetText']
  return tweetDict

#load your fishing tweet data here:
csvPathFish = "./tweetsFishing.csv"
rawTweetDictFish = loadTextFromCSV(csvPathFish)

#print your tweet dictionary. You should see your saved tweets inside.
print("rawTweetDictFish: ",rawTweetDictFish[0])

rawTweetDictFish:  Genuinely lovely episode of Gone Fishing this week. Bob and Paul just giddy messing about up at Loch Ness. Glorious.


## Task2

Doing segmentation of sentence and tokenize, lower-case all words and shuffle data then split with 80% train data and 20% test data.

In [4]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer
import random

def preprocess(rawTweetDataDict,ngram):
  #Input: a dictionary contains raw tweet data scraped from Tweeter

  #Output: two lists of tweet sentences (train/test), but each tweet sentence is
  #     represented in the form of tokens.
  def sentenceSegmentation(tweet):
    #Input: a string of raw tweet
    #Output: a list of strings, each element in the list is a segmented sentence
    sentences = sent_tokenize(tweet)
    return sentences
    
  def sentenceLowerCase(sentence):
    #Input: a string of sentence
    #Output: a string of sentence, but all words in the sentence are lower-cased.
    return sentence.lower()

  def sentenceTokenization(sentence):
    #Input: a string of sentence
    #Output: a list of tokens that belong to the sentence.
    tk = TweetTokenizer()
    return tk.tokenize(sentence)
  
  dataSet = []
  for i in range(len(rawTweetDataDict)):
    sentences = sentenceSegmentation(rawTweetDataDict[i])
    for sentence in sentences:
      lower_sentence = sentenceLowerCase(sentence)
      tokenizes = sentenceTokenization(lower_sentence)
      dataSet.append(tokenizes)
  random.shuffle(dataSet)
  offset = int(len(dataSet) * 0.8)
  return dataSet[0:offset], dataSet[offset:]


In [5]:
trainData, testData = preprocess(rawTweetDictFish,3)

In [6]:
print(trainData[0:5])
print(len(trainData))
print(testData[0:5])
print(len(testData))

[['i', 'had', 'a', 'co', 'worker', 'who', 'came', 'in', 'to', 'work', 'sun', 'burnt', 'as', 'hell', '…', 'i', 'was', 'like', '“', 'did', 'i', 'not', 'tell', 'you', 'don', '’', 't', 'take', 'yo', 'ass', 'out', 'there', 'fishing', '?'], ['no', 'matter', 'what', 'good', 'you', 'try', 'to', 'find', ',', 'the', 'only', 'thing', 'that', 'really', 'matters', 'is', 'the', 'win', '/', 'loss', 'column', '…', '.'], ['sigh', '...'], ['i', 'apologize', 'but', 'there', 'will', 'be', 'no', 'fox', 'stream', 'today', '.'], ['sometimes', 'fishing']]
11569
[["i'm", 'fishing', 'on', 'the', 'river', 'today', '🎣'], ['happy', 'national', 'hunting', '&', 'fishing', 'day', 'from', 'sportsman', '’', 's', 'guide', '!'], ['alright', ',', 'we', '’', 've', 'got', 'a', 'lawn', 'chair', 'for', 'eating', 'food', 'rations', 'in', ',', 'we', 'got', 'a', 'sleeping', 'bag', 'to', 'store', 'food', 'rations', 'in', ',', 'we', 'have', 'food', 'rations', 'for', 'eating', 'food', 'rations', '.'], ['uh', '...', 'all', 'of', 'th

## Task 3

Traning n-gram model with add-one smoothing

In [9]:
# Here's a template you may want to start with
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Laplace

def trainNGramAddOneSmoothing(trainData,ngram):
  # Input: a list of tweet sentences, each element is a list of tokens; n for ngram model
  
  train, vocab = padded_everygram_pipeline(order=ngram,text=testData)

  # lm = MLE(ngram)
  # ngram model with add-one smoothing
  lm = Laplace(ngram)
  lm.fit(train, vocab)
  # Output: a n-gram model with add-one smoothing trained on your input data. 
  return lm


In [10]:
triModel = trainNGramAddOneSmoothing(trainData,3)

In [11]:
triModel.generate(10,random_seed=5)

['just', 'some', 'of', 'you', 'may', 'have', 'heard', 'the', 'story', 'start']

In [12]:
biModel = trainNGramAddOneSmoothing(trainData,2)
biModel.generate(10,random_seed=5)

['marriage', '&', 'the', 'wailings', 'of', 'us', '!', '</s>', 'wild', 'petty']

In [13]:
uniModel = trainNGramAddOneSmoothing(trainData,1)
uniModel.generate(10,random_seed=5)

['not', 'shit', 'thank', 'with', "she's", 'well', ',', 'i', 'with', 'on']

## Task 4

Compute the average perplexity of my tri-gram model on the sentences of test data.

In [15]:
def computePerplexity(model,testData):
    # Input: your model; the testing data
    # Output: average perplexity of the model on your testing data.
    return model.perplexity(testData)


In [16]:
# compute 
# 1. the perplexity of your model on your testing data of 'fishing' tweets.
print(computePerplexity(triModel, testData))

6132.265913071364


In [17]:
# 2. the perplexity of your model on your data of 'football' tweets.
rawTweetDictFootball = loadTextFromCSV("./tweetsFootball.csv")
trainData_football, testData_football = preprocess(rawTweetDictFootball,3)
footballData = trainData_football + testData_football
print(computePerplexity(triModel, footballData))

6749.414881966336


**Why is there a difference between the two perplexities?**

First, there are different dataset that contains different data and words. The same model meets the same words and will predict the same result of the next word. But in different sentences, they have different true word after the input word, which will affect the value of conditional probability.  
Second, the dataset change from fishing to football, there will contain some words that are never included in the fishing data set. So, the model cannot recognize these words and would not generate a good prediction result.


## Task 5

Generating 10 tweets in 3 models

In [18]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
# Here's a template you may want to start with

def generateNewSentence(model,randomSeed):
  # Input: your model; random seed that get you different generated sentence
  sentence = model.generate(10, random_seed=randomSeed)
  detokenizer = TreebankWordDetokenizer()
  return detokenizer.detokenize(sentence)
  # Output: a new sentence generated by your model, but in a string format instead of tokens.

# Make loops to generate 10 tweets for each of your model (unigram, bigram and trigram)
print("The uni-gram result")
for i in range(10):
  print(generateNewSentence(uniModel,i))

print("The bi-gram result")
for i in range(10):
  print(generateNewSentence(biModel,i))

print("The tri-gram result")
for i in range(10):
  print(generateNewSentence(triModel,i))


The uni-gram result
this socrates guild community is go switched eating i'd me
act though south cocaine in hoo online talents 40,
yes women's ....these seriously paying engagement my my
can't launch fishing's my notice . #fishpuncontest they competition came
can; game and . getting we that spend brass
not shit thank with she's well, i with on
terrible the illustration confuse! over i somebody fly start
favourite an one . kids fishing . is, he
bust you a ref / chargers 🥶 better oh i
i focus afternoon to " interest valley . like next
The bi-gram result
the repair shop . </s> fish tanks " high-stick nymphing
</s> their product . </s> for prospering #oceans, 123
worst part . </s> the pursuit of great meetings with
already have gained their original, $500 + </s>
all because if i ask me too reading that don't
marriage & the wailings of us! </s> wild petty
ssn? </s> anyone? </s> go if it wasn
catch a person, it in a lil bit my
account specifically for sharks, cathy and doing some fishing
gear

## Task 6

In [19]:
# Here's a template you may want to start with

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from string import punctuation
# need to download 'stopwords' before using it.
# nltk.download('stopwords')
from nltk.corpus import stopwords
import re

def computeSentimentOfSentences(sentenceData, sentiment):
  # Input: a list of sentences from tweets
  Tweet = []
  sum = 0.0
  sid_obj = SentimentIntensityAnalyzer()
  for i in range(len(sentenceData)):
    # sentenceData[i] = removeStopWords(sentenceData[i])
    sentiment_dict = sid_obj.polarity_scores(sentenceData[i])
    sum += sentiment_dict['compound']
    if sentiment_dict['compound'] >= 0.05 and sentiment == 'positive':
      Tweet.append(sentenceData[i])
    if sentiment_dict['compound'] <= -0.05 and sentiment == 'negative':
      Tweet.append(sentenceData[i])
  # Output: a list of sentences from positive tweets, average compound from all the input sentences
  return Tweet, sum/len(sentenceData)

def removeStopWords(sentence):
  # Input: a sentence of tweet
  stop_words = set(stopwords.words('english'))
  filtered_sentence = [w for w in sentence if (w not in stop_words) and (w not in punctuation)]
  cleanSentence = []
  for word in filtered_sentence:
      tempWord = re.sub('(\W+)', '', word)
      if tempWord != '':
        cleanSentence.append(tempWord)
  # Output: the sentence of input tweet, but non-stop + non-punctuation words are removed
  return cleanSentence

1. Compute the ratios of positive and negative sentences in your collected data.


In [20]:
fishingData = trainData + testData
positiveList, averageCompoundFish = computeSentimentOfSentences(fishingData,'positive')
negativeList, averageCompoundFish = computeSentimentOfSentences(fishingData,'negative')
print("Positive ratio : " + str(len(positiveList) / len(fishingData) * 100) + "%")
print("Negative ratio : " + str(len(negativeList) / len(fishingData) * 100) + "%")

Positive ratio : 2.869589268427603%
Negative ratio : 1.9637671138155166%


2. compute the average compound of the collected data

In [21]:
positiveListFootball, averageCompoundFootball = computeSentimentOfSentences(footballData,'positive')
print("Average compound in fishing data : " + str(averageCompoundFish))
print("Average compound in football data : " + str(averageCompoundFootball))

Average compound in fishing data : 0.006148796846909132
Average compound in football data : 0.009423123443614376


I think they are generally neutral, but the average compound exceed 0 means there are more positive tweets than the negative tweets. So, maybe we can also think it is genrally positive.

3. Compute the top 10 non stop words from positive tweets of 'fishing'.

In [22]:
allWordsDict = {}
for sentence in positiveList:
    tempList = removeStopWords(sentence)
    for word in tempList:
        if word not in allWordsDict.keys():
            allWordsDict[word] = 0
        else:
            allWordsDict[word] += 1

In [23]:
top10 = sorted(allWordsDict.items(), key=lambda dict: dict[1], reverse=True)[0:10]
print('Here are the top 10 words below')
for i in range(10):
    print(top10[i][0])

Here are the top 10 words below
fishing
go
like
win
im
going
life
كود
خصم
ٱےهےربـ


The garbled words above are caused by the emoji during processing.